# LoRA Injection into SAM

In [1]:
# Step 0: Load SAM and inject LoRA weights into the mask decoder
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from src.sam_tools import load_sam_model
from src.lora_sam import inject_lora_into_sam_mask_decoder

predictor = load_sam_model(device="cuda")

sam = predictor.model

sam = inject_lora_into_sam_mask_decoder(sam)

# reassign to predictor
predictor.model = sam


D:\segment-anything\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


trainable params: 94,208 || all params: 4,152,548 || trainable%: 2.2687


In [1]:
# 1 Lora_train.py added
# Cell 1: Load dataset (Step-0)
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

from src.step2_ndvivariance import load_all_data

dataset = load_all_data()

print(f"Loaded {len(dataset)} images")

# Keep only images with GT masks
dataset = [item for item in dataset if item["mask"].sum() > 0]

print(f"Usable images (GT present): {len(dataset)}")


[Step-0] Image 1022: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0545
[Step-0] Image 1023: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0160
[Step-0] Image 1024: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0439
[Step-0] Image 1027: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0587
[Step-0] Image 1028: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0463
[Step-0] Image 1029: NDVI time-series loaded (T=6) | NDVI variance threshold=0.1050
[Step-0] Image 1031: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0850
[Step-0] Image 1032: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0618
[Step-0] Image 1099: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0321
[Step-0] Image 1100: NDVI time-series loaded (T=6) | NDVI variance threshold=0.0355
Loaded 10 images
Usable images (GT present): 8


In [3]:
# ============================================
# Debug run: LoRA training (1 epoch, 2 images)
# ============================================

from src.lora_train import train_lora
from src.config import RANKED_PROMPTS_DIR

# ---- Run LoRA training ----
sam_lora = train_lora(
    dataset=dataset,
    ranked_prompt_dir=RANKED_PROMPTS_DIR,
    epochs=1,
    lr=1e-4,
    device="cuda"
)

print("✅ Debug LoRA training finished")


[DEBUG] Using 2 images for LoRA sanity check


D:\segment-anything\segment_anything\build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


[LoRA] Trainable params: 94,208
[DEBUG] CUDA available: True
[DEBUG] SAM device: cuda:0
[DEBUG] Trainable LoRA params: 94208

[LoRA] Epoch 1/1
  → Image 1022 (1/8)
Available variables in NL_1022.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
     loss = 0.6791
  → Image 1024 (2/8)
Available variables in NL_1024.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
     loss = 1.3788
  → Image 1027 (3/8)
Available variables in NL_1027.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
     loss = 1.0600
  → Image 1028 (4/8)
Available variables in NL_1028.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
     loss = 1.7328
  → Image 1029 (5/8)
Available variables in NL_1029.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
     loss = 0.5227
  → Image 1031 (6/8)
Available variables in NL_1031.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
     loss = 5.2604
  → Image 1032 (7/8)
Available variables in NL_1032.nc: ['B2', 'B3', 'B4', 'B8', 'NDVI', 'spatial_ref']
     loss = 1.4